In [1]:
import pandas as pd 
import nltk 

In [2]:
data = pd.read_csv('train.csv')
data.drop('Id',axis=1 , inplace=True)
data

,Comment,Topic
0,A few things. You might have negative- frequen...,Biology
1,Is it so hard to believe that there exist part...,Physics
2,There are bees,Biology
3,I'm a medication technician. And that's alot o...,Biology
4,Cesium is such a pretty metal.,Chemistry
...,...,...
8690,I make similar observations over the last week...,Biology
8691,You would know.,Biology
8692,Also use the correct number of sig figs,Chemistry
8693,"What about the ethical delimmas, groundbreaki...",Biology


In [3]:
data.shape 
data.dtypes

Comment    object
Topic      object
dtype: object

In [4]:
CommentsToBeTokenized = data['Comment'] 
CommentsToBeTokenized

0       A few things. You might have negative- frequen...
1       Is it so hard to believe that there exist part...
2                                          There are bees
3       I'm a medication technician. And that's alot o...
4                          Cesium is such a pretty metal.
                              ...                        
8690    I make similar observations over the last week...
8691                                      You would know.
8692              Also use the correct number of sig figs
8693    What about the ethical delimmas,  groundbreaki...
8694                            I would like to know too.
Name: Comment, Length: 8695, dtype: object

In [5]:
tokens = CommentsToBeTokenized[0].split()
tokens

['A',
 'few',
 'things.',
 'You',
 'might',
 'have',
 'negative-',
 'frequency',
 'dependent',
 'selection',
 'going',
 'on',
 'where',
 'the',
 'least',
 'common',
 'phenotype,',
 'reflected',
 'by',
 'genotype,',
 'is',
 'going',
 'to',
 'have',
 'an',
 'advantage',
 'in',
 'the',
 'environment.',
 'For',
 'instance,',
 'if',
 'a',
 'prey',
 'animal',
 'such',
 'as',
 'a',
 'vole',
 'were',
 'to',
 'have',
 'a',
 'light',
 'and',
 'a',
 'dark',
 'phenotype,',
 'a',
 'predator',
 'might',
 'recognize',
 'the',
 'more',
 'common',
 'phenotype',
 'as',
 'food.',
 'So',
 'if',
 'the',
 'light',
 'voles',
 'are',
 'more',
 'common,',
 'foxes',
 'may',
 'be',
 'keeping',
 'a',
 'closer',
 'eye',
 'out',
 'for',
 'light',
 'phenotypic',
 'voles,',
 'recognising',
 'them',
 'as',
 'good',
 'prey.',
 'This',
 'would',
 'reduce',
 'the',
 'light',
 'causing',
 'alleles',
 'due',
 'to',
 'increased',
 'predation',
 'and',
 'the',
 'dark',
 'genotypes',
 'would',
 'increase',
 'their',
 'proport

In [6]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()
bow = cv.fit_transform(CommentsToBeTokenized)

In [7]:
bow

<8695x18177 sparse matrix of type '<class 'numpy.int64'>'
	with 190076 stored elements in Compressed Sparse Row format>

In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_result=tfidf.fit_transform(CommentsToBeTokenized).toarray()

In [9]:
tfidf_result.shape

(8695, 18177)

In [10]:
vectorizer = TfidfVectorizer(max_features=18177)
tfidf_result = vectorizer.fit_transform(CommentsToBeTokenized).toarray()
features = vectorizer.get_feature_names_out()
tfidf_result = pd.DataFrame(tfidf_result, columns=features)

In [11]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
NB= MultinomialNB()

In [18]:
xtrain, xvalid, ytrain, yvalid = train_test_split(tfidf_result, data['Topic'], test_size= 0.2)
NB.fit(xtrain, ytrain)

MultinomialNB()

In [19]:
ypred_train = NB.predict(xtrain)
ypred = NB.predict(xvalid)

In [16]:
ypred.shape

(1739,)

In [20]:
print("Training Results:\n")
print(classification_report(ytrain, ypred))
print("\nValidation Results:\n")
print(classification_report(ytest, ypred))

Training Results:



ValueError: Found input variables with inconsistent numbers of samples: [6956, 1739]

In [5]:
df = data.groupby('Topic')

In [7]:
df.count()

,Comment
Topic,
Biology,3591
Chemistry,2920
Physics,2184
